# Demo for paper "First Order Motion Model for Image Animation"

**Mount Google Drive into colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Runtime Details**

In [ ]:
%cat /etc/lsb-release

**Go inside the deepfake folder of google drive**

In [ ]:
%cd /content/drive/My\ Drive/deepfake

**Clone repository**

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

**Check current directory**

In [ ]:
%pwd

**Go inside the first order model folder**

In [ ]:
cd first-order-model

**Clone face-alignment library**

In [ ]:
!git clone https://github.com/1adrianb/face-alignment

**Go inside face-alignment**

In [ ]:
%cd face-alignment/

**Install the requirements for face-alignment**

In [ ]:
!pip install -r requirements.txt

**Install face-alignment library**

In [ ]:
!python setup.py install

**Check the current directory**

In [ ]:
%pwd

**Go one directory to the back**

In [ ]:
%cd ..

**List the contents of directory**

In [ ]:
%ls

**Generate the command to do the driving video crop**

In [ ]:
!python crop-video.py --inp samples/driving_video_sample1.mp4

**Run the ffmpeg command to do the cropping**

In [ ]:
!ffmpeg -i samples/driving_video_sample1.mp4 -ss 0.0 -t 49.166666666666664 -filter:v "crop=445:445:435:80, scale=256:256" crop.mp4

**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Run the animation using command from the github repo**

In [ ]:
!python demo.py  --config config/vox-adv-256.yaml --driving_video samples/crop.mp4 --source_image samples/03.png --checkpoint samples/vox-adv-cpk.pth.tar --relative --adapt_scale

**Load driving video and source image**

In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/04.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [ ]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [ ]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [ ]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/07.mkv -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 hinton.mp4

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.** 

In [ ]:
source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/09.png')
driving_video = imageio.mimread('hinton.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())